In [51]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [52]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(
    Path('Resources/lending_data.csv')
)

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [53]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df['loan_status']
# Separate the X variable, the features
X = df.drop(columns='loan_status')

In [54]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [55]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [56]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [57]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [58]:
#Scaling the Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [59]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [60]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test_scaled)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [61]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9889115309798473

In [62]:
# Generate a confusion matrix for the model
confusion_matrix(predictions,y_test)

array([[18652,    10],
       [  113,   609]], dtype=int64)

In [63]:
# Print the classification report for the model
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     18662
           1       0.98      0.84      0.91       722

    accuracy                           0.99     19384
   macro avg       0.99      0.92      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Although the algorithm is slightly better at predicting healthy loans than the high risk loans, the f1-score for predicting risky loans is still ~91%, which makes the algorithm a strong predicter of loan outcome.


---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [64]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_res, y_res = ros.fit_resample(X_train_scaled, y_train)


In [65]:
# Count the distinct values of the resampled labels data
print(pd.DataFrame(X_res).value_counts())
print(pd.DataFrame(y_res).value_counts())

0         1         2         3         4         5         6       
4.152438  4.140225  4.141294  3.255755  4.292342  2.759592  4.141294    209
3.674708  3.663539  3.663474  3.035800  3.767093  2.759592  3.663474    181
3.626935  3.616321  3.615692  3.012587  3.767093  2.759592  3.615692    157
4.104665  4.098628  4.093512  3.234707  4.292342  2.759592  4.093512    153
4.534622  4.537089  4.535496  3.422026  4.292342  2.759592  4.535496    140
                                                                       ... 
1.477150  1.462239  1.465503  1.655489  1.666098  1.043347  1.465503      1
          1.466736  1.465503  1.655489  1.666098  1.043347  1.465503      1
          1.470109  1.465503  1.655489  1.666098  1.043347  1.465503      1
          1.476855  1.477449  1.665219  1.666098  1.043347  1.477449      1
6.684407  6.679928  6.685685  4.145331  6.393338  4.475838  6.685685      1
Length: 4634, dtype: int64
loan_status
0              56271
1              56271
dtype: int64


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [66]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier2 = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier2.fit(X_res, y_res)

# Make a prediction using the testing data
predictions2 = classifier2.predict(X_test_scaled)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [67]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, predictions2)

0.9934383134311076

In [68]:
# Generate a confusion matrix for the model
confusion_matrix(predictions2, y_test)

array([[18640,     4],
       [  125,   615]], dtype=int64)

In [69]:
# Print the classification report for the model
print(classification_report(predictions2, y_test))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     18644
           1       0.99      0.83      0.91       740

    accuracy                           0.99     19384
   macro avg       0.99      0.92      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Compared to the original scaled model, the oversampled model very slightly improves on accuracy, 98.9% tp 99.3%. The f1 score for risky loans ultimately remains the same at 91%. 